In [1]:
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import AdamW
from transformers import BertForSequenceClassification

class Dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]

token = BertTokenizer.from_pretrained('bert-base-chinese')

In [2]:
def collate_fn(data):
    sents = [data[i][0] for i in range(len(data))]
    labels = [data[i][1] for i in range(len(data))]
    
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                    truncation=True, 
                    padding='max_length',
                    max_length=50,
                    return_tensors='pt',
                    return_length=True)
    
    input_ids = data['input_ids'] 
    attention_mask = data['attention_mask'] 
    token_type_ids = data['token_type_ids'] 
    labels = torch.LongTensor(labels) 
    
    return input_ids, attention_mask, token_type_ids, labels

In [3]:
model = torch.load('2022.5.4_5_model.pth',map_location ='cpu')

In [10]:
test_dataset = Dataset(["陳伯彥跟鬼一樣"], [7414]) #0是喜 1是怒 2是哀
testloader = DataLoader(dataset=test_dataset, batch_size=1, collate_fn=collate_fn)
for i in testloader:
    tokens_tensors, segments_tensors, masks_tensors = i[:3]
    outputs = model(input_ids=tokens_tensors, 
          token_type_ids=segments_tensors, 
          attention_mask=masks_tensors)
    logits = outputs[0]
    labels = i[3]
    _, pred = torch.max(logits.data, 1)
    if pred[0] == 0:
        print(pred[0], '喜')
    elif pred[0] == 1:
        print(pred[0], '怒')
    elif pred[0] == 2:
        print(pred[0], '哀')

tensor(0) 喜
